In [1]:
import pandas as pd
from urllib.parse import urlparse

In [2]:
csv_path = '/media/joeymeyer/860-evo-ubuntu/Data/caves_org/caves_real.csv'
csv_path_analysis = '/media/joeymeyer/860-evo-ubuntu/Data/caves_org/caves_analysis.csv'
df = pd.read_csv(csv_path)
df = df.drop(columns=['Unnamed: 0'])
df.shape

(19104860, 9)

In [3]:
df.head()

,domain,port,ip_address,date_time,http_request,response_code,response_bytes,referrer,user_agent
0,caves.org,443,103.230.123.233,02/Feb/2020:06:42:07 -0500,GET /region/rmr/index.php/contact HTTP/1.1,503,8214,https://caves.org/region/rmr/index.php/contact,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
1,caves.org,443,100.36.204.30,02/Feb/2020:06:42:08 -0500,GET /images/webads/hodagvideo.jpg HTTP/1.1,200,15240,https://caves.org/,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
2,caves.org,443,185.245.40.24,02/Feb/2020:06:42:07 -0500,GET /region/rmr/index.php/contact HTTP/1.1,503,8214,https://caves.org/region/rmr/index.php/contact,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...
3,caves.org,443,185.245.40.7,02/Feb/2020:06:42:07 -0500,POST /region/rmr/index.php/contact HTTP/1.1,303,6183,https://caves.org/region/rmr/index.php/contact,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...
4,caves.org,443,198.44.179.32,02/Feb/2020:06:42:07 -0500,POST /region/rmr/index.php/contact HTTP/1.1,303,6183,https://caves.org/region/rmr/index.php/contact,Mozilla/5.0 (Windows NT 6.3; Win64; x64) Apple...


The cells below split the http_request column into multiple columns, removes the query request from the url, and further filters the data to only look at successful get requests, and removes extraneous information that loads each time a website page is visited.

In [4]:
request = df.pop('http_request').str.split()
df['resource'] = request.str[1]
df = df[(request.str[0] == 'GET') & (df.pop('response_code') == 200)]

In [5]:
df = df[~df['resource'].str.match(
    r'^/media|^/static|^/admin|.*image.*|.*phpmyadmin.*|^/robots.txt$|.*favicon.ico$|.*jpg$|.*png$|.*css$|.*gif$|.*\.js.*|.*graphics.*')]

In [6]:
df['resource'] = df['resource'].apply(lambda x: urlparse(x).path)
df

,domain,port,ip_address,date_time,response_bytes,referrer,user_agent,resource
5,caves.org,443,50.205.244.12,02/Feb/2020:06:42:09 -0500,28282,-,Mozilla/5.0 (compatible; PRTG Network Monitor ...,/
13,caves.org,443,93.114.234.116,02/Feb/2020:06:42:11 -0500,10882,-,Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:62....,/grotto/flittermouse/wp-login.php
17,caves.org,80,82.131.106.39,02/Feb/2020:06:42:14 -0500,1498,-,Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_5...,/section/vertical/nh/50/knotrope.html
28,caves.org,80,82.131.106.39,02/Feb/2020:06:42:16 -0500,1497,-,Mozilla/5.0 (iPhone; CPU iPhone OS 10_0 like M...,/section/vertical/nh/50/knotrope.html
44,caves.org,80,50.205.244.12,02/Feb/2020:06:42:49 -0500,20495,-,Mozilla/5.0 (compatible; PRTG Network Monitor ...,/
...,...,...,...,...,...,...,...,...
19104842,caves.org,443,50.205.244.12,24/Nov/2019:06:43:28 -0500,27913,-,Mozilla/5.0 (compatible; PRTG Network Monitor ...,/
19104843,caves.org,443,24.254.11.169,24/Nov/2019:06:43:44 -0500,11899,https://www.google.com/,Mozilla/5.0 (iPhone; CPU iPhone OS 12_4_1 like...,/grotto/mongrotto/clover_hollow.htm
19104852,caves.org,443,24.254.11.169,24/Nov/2019:06:44:23 -0500,43949,https://caves.org/grotto/mongrotto/clover_holl...,Mozilla/5.0 (iPhone; CPU iPhone OS 12_4_1 like...,/grotto/mongrotto/index.html
19104858,caves.org,80,50.205.244.12,24/Nov/2019:06:44:23 -0500,20242,-,Mozilla/5.0 (compatible; PRTG Network Monitor ...,/


In [7]:
df['ip_address'].value_counts()[:5].divide(df.ip_address.count())

50.205.244.12     0.207657
73.20.126.64      0.004231
67.163.77.242     0.003480
34.240.174.214    0.003290
34.251.10.88      0.002830
Name: ip_address, dtype: float64

In [8]:
df = df[df['ip_address'] != "50.205.244.12"]
df.shape

(3835335, 8)

The IP address above accounts for 20% of website traffic. This is suspiciously high and is likely a crawler. With that removed, the dataframe is now ready for analysis.

In [9]:
df.to_csv(csv_path_analysis)